## Importação de bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Concatenação de dados

In [ ]:
def load_and_concat_data_from_csv(data: list) -> pd.DataFrame:
    """
    documentation
    """
    dataframes = []
    for file in data:
        df = pd.read_csv(file)

        ## Esses dados "lon" são inuteis e não deveriam estar no dataset
        df = df[df["resource"] != "lon"]

        dataframes.append(df)
    
    return pd.concat(dataframes)

In [ ]:
df_itu_415_csv_locations = ["../../data/full_history_ITU-415_2025-06-01_a_2025-06-17.csv", "../../data/full_history_ITU-415_2025-06-17_a_2025-06-29.csv",
                            "../../data/full_history_ITU-415_2025-06-29_a_2025-06-31.csv", "../../data/full_history_ITU-415_2025-07-01_a_2025-07-17.csv"]

df_itu_693_csv_locations = ["../../data/full_history_ITU-693_2025-05-01_a_2025-06-08.csv", "../../data/full_history_ITU-693_2025-06-08_a_2025-07-22.csv"]

In [ ]:
df_itu_415 = load_and_concat_data_from_csv(df_itu_415_csv_locations)
df_itu_693 = load_and_concat_data_from_csv(df_itu_693_csv_locations)

In [ ]:
df_itu_415.head(3)

In [ ]:
df_itu_693.head(3)

# Features

| Tópico | Recurso | Código | Descrição | Unidade |
|--------|----------|--------|-----------|---------|
| Motor | Pressão de Óleo | %GROUP%/%SITE%/%UID%/Eng/Oil_P | Leitura de Pressão do Óleo – Pressão de Óleo no Motor | mca |
| Motor | Nível de Óleo | %GROUP%/%SITE%/%UID%/Eng/Oil_L | Leitura de Nível do Óleo – Nível de Óleo do Motor | % |
| Motor | Temp. Arrefecimento | %GROUP%/%SITE%/%UID%/Eng/Cool_T | Leitura Temperatura Líquido Arrefecimento – Temperatura do líquido de arrefecimento | °C |
| Motor | Nível Combustível | %GROUP%/%SITE%/%UID%/Eng/Fuel_L | Leitura Nível Combustível (%) – Nível de combustível no tanque do motor | % |
| Motor | Tensão Alternador | %GROUP%/%SITE%/%UID%/Eng/Char_V | Leitura Voltagem do Alternador – Tensão no alternador | V |
| Motor | Tensão Bateria | %GROUP%/%SITE%/%UID%/Eng/Bat_V | Leitura Voltagem da Bateria – Tensão na Bateria | V |
| Motor | Rotação Motor | %GROUP%/%SITE%/%UID%/Eng/Eng_RPM | Leitura de Rotações do Motor (RPM) – Rotação do motor | RPM |
| Motor | Consumo Combustível | %GROUP%/%SITE%/%UID%/Eng/Fuel_Con | Consumo de Combustível – Consumo de combustível do motor (**não usada**) | Não usada |
| Motor | Horímetro | %GROUP%/%SITE%/%UID%/Eng/EngRT_H | Horímetro (Segundos) – Tempo de operação da máquina | Seg. |
| Motor | Número de Partidas | %GROUP%/%SITE%/%UID%/Eng/Starts_N | Número de partidas – Quantidade de vezes em que a máquina foi ligada | Un. |
| Bomba | Pressão Recalque | %GROUP%/%SITE%/%UID%/Pump/Recalque | Sensor de Recalque – Sensor da pressão de descarga (saída) da bomba centrífuga | mca |
| Bomba | Vibração | %GROUP%/%SITE%/%UID%/Pump/Vibracao | Sensor de Vibração – Vibração da centrífuga (**não usada**) | Não usada |
| Bomba | Sucção 1 | %GROUP%/%SITE%/%UID%/Pump/FlexAnalogue6_1 | Sensor de Sucção – Sensor de sucção (entrada) da bomba centrífuga | mca |
| Bomba | Sucção 2 | %GROUP%/%SITE%/%UID%/Pump/FlexAnalogue6_1 | Sensor de Sucção – Sensor de sucção (entrada) da bomba centrífuga | mca |
| Bomba | Sucção 3 | %GROUP%/%SITE%/%UID%/Pump/FlexAnalogue6_2 | Sensor de Sucção – Sensor de sucção (entrada) da bomba centrífuga | mca |
| Bomba | Sucção 4 | %GROUP%/%SITE%/%UID%/Pump/FlexAnalogue7_1 | Sensor de Sucção – Sensor de sucção (entrada) da bomba centrífuga | mca |
| Painel | LED Stop | %GROUP%/%SITE%/%UID%/LED/Stop | Read for stop mode LED – Indicação de máquina parada (0 = operando; 1 = parada) | On/off |
| Painel | LED Manual | %GROUP%/%SITE%/%UID%/LED/Man | Read for manual mode LED – Modo manual do painel de controle | On/off |
| Painel | LED Auto | %GROUP%/%SITE%/%UID%/LED/Auto | Read for auto mode LED – Modo automático do painel de controle | On/off |

## Verificar quantidade de registros

In [ ]:
print(f"Quantidade de dados no df_itu_415: {df_itu_415.shape[0]}")
print(f"Quantidade de dados no df_itu_693: {df_itu_693.shape[0]}")

In [ ]:
df_itu_693["resource"].value_counts()

## Frequência de coleta de dados

## Frequência de coleta de cada um dos dados

In [ ]:
def calculate_frequency_stats(df: pd.DataFrame, motor_pump: str) -> pd.DataFrame:
    """Calculate per-resource inter-event time statistics (in seconds).

    For each distinct resource in df['resource'], this function:
    1. sorts rows by the timestamp column,
    2. computes the time difference between consecutive timestamps with
       Series.diff(),
    3. converts the resulting timedeltas to seconds with .dt.total_seconds(),
    4. drops the first NaN difference (there is no previous row to subtract),
    5. and aggregates common descriptive statistics (count, mean, median,
       std, min, max and the 25th/75th percentiles).

    Args:
        df (pandas.DataFrame): Input DataFrame containing at least the
            following columns:
                - timestamp (datetime-like): event timestamps; must be
                  parseable/convertible by pandas (timezone-aware or naive).
                - resource (str): resource identifier used to group rows.
        motor_pump (str): Identifier to set in the output column motor_pump
            (for example, "ITU-415" or "ITU-693").

    Returns:
        pandas.DataFrame: A DataFrame with one row per resource and the
        following columns:
            - motor_pump (str): value passed via motor_pump argument.
            - resource (str): resource identifier.
            - count (int): number of computed intervals (N - 1 for N rows).
            - mean_interval (float): mean inter-event interval (seconds).
            - median_interval (float): median inter-event interval (seconds).
            - std_interval (float): standard deviation of intervals (seconds).
            - min_interval (float): minimum interval (seconds).
            - max_interval (float): maximum interval (seconds).
            - q25_interval (float): 25th percentile interval (seconds).
            - q75_interval (float): 75th percentile interval (seconds).

    Raises:
        TypeError: If df is not a pandas.DataFrame.
        ValueError: If required columns ('timestamp' or 'resource') are
            missing from df.
        TypeError: If the 'timestamp' column cannot be interpreted as
            datetime-like values by pandas.

    Notes:
        - The first row per resource group will not produce an interval because
          ``diff()`` returns NaN for the first element; that NaN is intentionally
          dropped before computing statistics.
        - Interval conversion uses ``.dt.total_seconds()`` so all returned
          interval statistics are expressed in seconds. For sub-second
          precision this returns floating-point values (fractions of a second).
        - If you need wall-clock-aware comparisons across timezones, ensure
          timestamps are normalized or timezone-aware before calling this
          function."""
    
    ## Fazemos isso para possibilitar fazer operações com data
    ## format=ISO8601 porque o formato das datas não é constante, mas é em ISO8601
    df["timestamp"] = pd.to_datetime(df["timestamp"], format="ISO8601")

    resources = df["resource"].unique()
    frequency_stats = []
    
    for resource in resources:
        df_resource = df[df["resource"] == resource].copy()
        df_resource = df_resource.sort_values(by="timestamp")
        
        df_resource["time_diff"] = df_resource["timestamp"].diff()
        df_resource["time_diff_secs"] = df_resource["time_diff"].dt.total_seconds()
        
        ## Como o diff subtrai o valor da colune anterior,
        ## a primeira linha sempre vai ser NaN.
        time_diffs = df_resource["time_diff_secs"].dropna()
        
        if len(time_diffs) > 0: 
            stats = {
                'motor_pump': motor_pump,
                'resource': resource,
                'count': len(time_diffs),
                'mean_interval': time_diffs.mean(),
                'median_interval': time_diffs.median(),
                'std_interval': time_diffs.std(),
                'min_interval': time_diffs.min(),
                'max_interval': time_diffs.max(),
                'q25_interval': time_diffs.quantile(0.25),
                'q75_interval': time_diffs.quantile(0.75)
            }
        
        frequency_stats.append(stats)
    
    return pd.DataFrame(frequency_stats)

freq_stats_415 = calculate_frequency_stats(df_itu_415, "ITU-415")
freq_stats_693 = calculate_frequency_stats(df_itu_693, "ITU-693")

all_freq_stats = pd.concat([freq_stats_415, freq_stats_693], ignore_index=True)

all_freq_stats_sorted = all_freq_stats.sort_values('mean_interval')


In [ ]:
all_freq_stats_sorted.head()

In [ ]:
plt.figure(figsize=(15, 8))

valid_stats = all_freq_stats_sorted[all_freq_stats_sorted['mean_interval'].notna()]

motor_pumps = valid_stats['motor_pump'].unique()
colors = ['blue', 'red']

for i, motor_pump in enumerate(motor_pumps):
    motor_pump_data = valid_stats[valid_stats['motor_pump'] == motor_pump]
    
    plt.subplot(1, 2, i+1)
    plt.barh(range(len(motor_pump_data)), motor_pump_data['mean_interval'], color=colors[i], alpha=0.7)
    plt.yticks(range(len(motor_pump_data)), motor_pump_data['resource'], fontsize=8)
    plt.xlabel('Intervalo Médio em segundos')
    plt.title(f'Frequência de Coleta da bomba {motor_pump}')
    plt.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## Conclusões sobre frequência: 

* Para o ITU415, dados são medidos e enviados, em média, a cada 12,17s
* Para ITU693, dados são medidos e enviados, em média, a cada 7,23s
* Ambos os casos possuem alguns outliers, como tempo entre medidas de quase 500.000 segundos
* ITU415 possui mais tipos de dados
* Eng_RPM, Recalque e Succao são medidos no mesmo intervalo
* Nem todos os dados são medidos no mesmo intervalo

## Qualidade dos dados

Verificar valores nulos, duplicados, fora de faixa possível, outliers, etc

In [ ]:
df_itu_415.isna().sum()

In [ ]:
df_itu_693.isna().sum()

In [ ]:
print(f"Quantidade de dados duplicados em ITU415: {len(df_itu_415[df_itu_415.duplicated()])}")
print(f"Quantidade de dados duplicados em ITU693: {len(df_itu_693[df_itu_693.duplicated()])}")

In [ ]:
df_itu_693[df_itu_693.duplicated()]

In [ ]:
df_itu_693.shape

In [ ]:
df_itu_415[df_itu_415["resource"] == "Bat_V"].describe()

### Conclusões: 

- Quantidade de dados duplicados em ITU415: 55856
- Quantidade de dados duplicados em ITU693: 1673724

Nulos no 415:
- resource: 49
- value: 36

Nulos no 693:
- resource: 1182
- value: 1126

Valores fora do padrão: 
- Valores de bateria aparentemente precisam ser divididos por 10
- Valores muito altos (>2bi) devem ser considerados como 0 e criar uma flag de running



## Limpeza

Remoção de nulos, duplicados e ajuste de valores gigantescos

In [ ]:
def remove_duplicates_and_nan(df: pd.DataFrame) -> pd.DataFrame:
    df_cleaned = df.drop_duplicates().dropna()
    return df_cleaned

In [ ]:
remove_duplicates_and_nan(df_itu_415)
remove_duplicates_and_nan(df_itu_693)

Tratar valores 2bi e criar coluna nova

In [ ]:
def treat_high_values(df: pd.DataFrame, max_limit: int) -> None:
    df['running'] = np.where(df['value'] > max_limit, 0, 1)
    df['value'] = np.where(df['value'] > max_limit, 0, df['value'])

In [ ]:
treat_high_values(df = df_itu_693, max_limit=20000)
treat_high_values(df = df_itu_415, max_limit=20000)

In [ ]:
df_itu_693.head(3)

In [ ]:
df_itu_693[df_itu_693["value"] > 3000].head(3)

In [ ]:
def fix_battery_and_alternator_values(df: pd.DataFrame) -> None:
    df.loc[df["resource"] == "Bat_V", "value"] = df.loc[df["resource"] == "Bat_V", "value"] / 10
    df.loc[df["resource"] == "Char_V", "value"] = df.loc[df["resource"] == "Char_V", "value"] / 10

In [ ]:
fix_battery_and_alternator_values(df_itu_415)
fix_battery_and_alternator_values(df_itu_693)

## Transformação de long para wide

Criar colunas para cada tipo de resource diferente

In [ ]:
def pivot_df(df: pd.DataFrame, resample_seconds: int = 60) -> pd.DataFrame:
    """Converts motor pump data from long to wide format and resamples temporally.

    This function transforms the DataFrame from long format (one row per sensor reading)
    to wide format, creates one column per `resource`, adds the `running` column, and
    resamples the data for each pump (`motor_pump`) in time windows (e.g., 80 seconds).

    For each pump (motor_pump), the function resamples the data in regular time windows
    (e.g., every 80 seconds). The `resample` method groups the data into these fixed
    intervals, computing the mean of the values within each period. After resampling,
    forward fill (ffill) is used to fill missing values, and the 'running' column is
    adjusted to be binary (0 or 1).

    Steps:
        1. Pivot: transforms each `resource` into a column. If there are multiple
           readings at the same instant, calculates the mean.
        2. Merge: adds the `running` column to the wide DataFrame.
        3. Resampling: for each pump, data is aggregated in 80s windows,
           filling missing values with forward fill and ensuring `running`
           is only 0 or 1.

    Args:
        df (pd.DataFrame): Long DataFrame with columns `timestamp`, `motor_pump`,
            `resource`, `value`, and `running`.
        resample_seconds: Int type value that represents the amount of time, in seconds,
        that will be considerated for the resampling function

    Returns:
        pd.DataFrame: Wide DataFrame, resampled every 80s, with one column per
        resource, binary `running` column, and columns `timestamp` and `motor_pump`."""
    df_wide = (
        df.pivot_table(
            index=["timestamp", "motor_pump"],
            columns="resource",
            values="value",
            aggfunc="mean"
        )
        .reset_index()
    )

    df_running = df[["timestamp", "motor_pump", "running"]]
    df_wide = df_wide.merge(df_running, on=["timestamp", "motor_pump"], how="left")
    df_wide = df_wide.set_index("timestamp")

    resampled = []
    for pump_id, group in df_wide.groupby("motor_pump"):
        g = (
            group
            .resample(f"{resample_seconds}s")
            .mean(numeric_only=True)
            .ffill()
        )
        g["running"] = g["running"].round().astype(int)
        g["motor_pump"] = pump_id
        resampled.append(g)

    df_wide = pd.concat(resampled).reset_index()
    return df_wide.drop_duplicates().fillna(df.mode().iloc[0])


In [ ]:
df_itu_693_wide = pivot_df(df=df_itu_693, resample_seconds=5)
df_itu_415_wide = pivot_df(df=df_itu_415, resample_seconds=5)

## Dataframes após transformar resources em features

In [ ]:
df_itu_693_wide.head(3)

In [ ]:
df_itu_415_wide.head(3)

In [ ]:
print(f"Quantidade de linhas após wide no ITU415: {len(df_itu_415_wide)}")
print(f"Quantidade de linhas após wide no ITU693: {len(df_itu_693_wide)}")

## Exploração gráfica

In [ ]:
def generate_time_series_graphics(df: pd.DataFrame) -> None:
    num_cols = df.select_dtypes(include='float64').columns.tolist()

    sns.set_theme(style="whitegrid")
    plt.rcParams['figure.figsize'] = (16, 2.5 * len(num_cols))

    fig, axes = plt.subplots(len(num_cols), 1, sharex=True, figsize=(16, 2.5 * len(num_cols)))

    for i, col in enumerate(num_cols):
        ax = axes[i]
        sns.lineplot(data=df, x='timestamp', y=col, hue='motor_pump', legend=False, ax=ax)
        ax.set_title(f'Série temporal - {col}')
        ax.set_xlabel('')
        ax.set_ylabel(col)

    axes[-1].set_xlabel('Timestamp')
    plt.tight_layout()
    plt.show()

In [ ]:
## generate_time_series_graphics(df_itu_415_wide)

In [ ]:
## generate_time_series_graphics(df_itu_693_wide)

## Com os gráficos acima, é possível identificar dados que estão sempre zerados e descartá-los

In [ ]:
def remove_columns_with_only_zeroes(df: pd.DataFrame) -> pd.DataFrame:
    """Removes columns from a DataFrame that contain only zero values.

    Args:
        df (pd.DataFrame): Input DataFrame to process.

    Returns:
        pd.DataFrame: DataFrame with columns containing only zero values removed."""
    zero_columns = (df == 0).all()
    columns_to_keep = zero_columns[~zero_columns].index
    df_cleaned = df[columns_to_keep]
    return df_cleaned


In [ ]:
df_itu_415_wide = remove_columns_with_only_zeroes(df_itu_415_wide)
df_itu_693_wide = remove_columns_with_only_zeroes(df_itu_693_wide)

In [ ]:
df_itu_415_wide.head(3)

In [ ]:
df_itu_693_wide.head(3)

In [ ]:
df_itu_693_wide.describe()

## ----------------------------------------------------------------------------------------------------

In [ ]:
df_test_raw = load_and_concat_data_from_csv(df_itu_693_csv_locations)
df_test_raw["timestamp"] = pd.to_datetime(df_test_raw["timestamp"], format="ISO8601")
df_cleaned = remove_duplicates_and_nan(df_test_raw)
treat_high_values(df=df_cleaned, max_limit=20000)
fix_battery_and_alternator_values(df=df_cleaned)
df_test_wide = pivot_df(df=df_cleaned, resample_seconds=15)
df_test_wide = remove_columns_with_only_zeroes(df_test_wide)


In [ ]:
df_test_wide.head(5)

In [ ]:
generate_time_series_graphics(df_test_wide)